In [1]:
import tensorflow as tf

/home/krohak/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from utils import *

In [3]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('dataset', 'pubmed', 'Dataset string.')  # 'cora', 'citeseer', 'pubmed'
flags.DEFINE_string('model', 'gcn', 'Model string.')  # 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 16, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 10, 'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

In [4]:
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(FLAGS.dataset)

In [39]:
print(adj[0])

  (0, 1378)	1
  (0, 1544)	1
  (0, 6092)	1
  (0, 7636)	1
  (0, 14442)	1


In [45]:
import networkx as nx
G = nx.Graph(adj)
#G.add_path([0,1,2])
#list(nx.bfs_edges([0,1,2],0))

In [55]:
print(list(nx.bfs_tree(G,0,0)))

[0, 1378, 1544, 6092, 7636, 14442, 99, 678, 704, 1720, 3541, 4048, 4203, 6145, 6733, 7443, 8302, 8602, 9234, 10259, 10728, 11142, 11651, 12160, 12643, 12746, 12925, 12972, 13272, 14502, 14770, 14896, 15314, 15503, 16217, 16527, 16822, 17277, 17687, 19349, 1233, 1290, 2078, 2110, 4976, 6106, 6769, 7379, 8931, 9809, 9861, 10087, 10088, 10216, 10740, 10976, 12280, 12360, 13662, 16684, 16976, 17102, 17412, 18242, 18459, 18814, 18855, 1728, 2344, 7191, 7297, 10665, 12103, 15099, 16772, 17054, 607, 1222, 1239, 1308, 1800, 3429, 4689, 5467, 7465, 8140, 12234, 12668, 13544, 14276, 16702, 17392, 17896, 18133, 18555, 314, 884, 7012, 7798, 7851, 8117, 8372, 8648, 9985, 10121, 11482, 12340, 15129, 16608, 357, 1839, 2874, 3286, 5430, 9192, 10282, 14216, 17298, 18823, 1469, 12457, 15882, 869, 5225, 7536, 13256, 16918, 411, 2090, 2673, 4911, 6568, 8192, 8857, 12281, 14149, 15885, 16004, 16513, 18745, 18960, 9890, 6296, 15458, 4402, 11139, 13876, 16054, 13263, 15088, 16839, 18030, 18367, 12924, 15857,

In [73]:
print(nx.shortest_path_length(G,source=0))

{0: 0, 1378: 1, 1544: 1, 6092: 1, 7636: 1, 14442: 1, 99: 2, 678: 2, 704: 2, 1720: 2, 3541: 2, 4048: 2, 4203: 2, 6145: 2, 6733: 2, 7443: 2, 8302: 2, 8602: 2, 9234: 2, 10259: 2, 10728: 2, 11142: 2, 11651: 2, 12160: 2, 12643: 2, 12746: 2, 12925: 2, 12972: 2, 13272: 2, 14502: 2, 14770: 2, 14896: 2, 15314: 2, 15503: 2, 16217: 2, 16527: 2, 16822: 2, 17277: 2, 17687: 2, 19349: 2, 1233: 2, 1290: 2, 2078: 2, 2110: 2, 4976: 2, 6106: 2, 6769: 2, 7379: 2, 8931: 2, 9809: 2, 9861: 2, 10087: 2, 10088: 2, 10216: 2, 10740: 2, 10976: 2, 12280: 2, 12360: 2, 13662: 2, 16684: 2, 16976: 2, 17102: 2, 17412: 2, 18242: 2, 18459: 2, 18814: 2, 18855: 2, 1728: 2, 2344: 2, 7191: 2, 7297: 2, 10665: 2, 12103: 2, 15099: 2, 16772: 2, 17054: 2, 607: 2, 1222: 2, 1239: 2, 1308: 2, 1800: 2, 3429: 2, 4689: 2, 5467: 2, 7465: 2, 8140: 2, 12234: 2, 12668: 2, 13544: 2, 14276: 2, 16702: 2, 17392: 2, 17896: 2, 18133: 2, 18555: 2, 314: 2, 884: 2, 7012: 2, 7798: 2, 7851: 2, 8117: 2, 8372: 2, 8648: 2, 9985: 2, 10121: 2, 11482: 2, 1

In [82]:
one_path = nx.all_pairs_shortest_path(G,cutoff=1)[0]
two_path = nx.all_pairs_shortest_path(G,cutoff=2)[0]

In [87]:
print(len(two_path), len(one_path)-1, len(two_path)-len(one_path)-1)

109 5 102


In [88]:
one_path

{0: [0],
 1378: [0, 1378],
 1544: [0, 1544],
 6092: [0, 6092],
 7636: [0, 7636],
 14442: [0, 14442]}

In [89]:
two_path

{0: [0],
 99: [0, 1378, 99],
 314: [0, 14442, 314],
 607: [0, 7636, 607],
 678: [0, 1378, 678],
 704: [0, 1378, 704],
 884: [0, 14442, 884],
 1222: [0, 7636, 1222],
 1233: [0, 1544, 1233],
 1239: [0, 7636, 1239],
 1290: [0, 1544, 1290],
 1308: [0, 7636, 1308],
 1378: [0, 1378],
 1544: [0, 1544],
 1720: [0, 1378, 1720],
 1728: [0, 6092, 1728],
 1800: [0, 7636, 1800],
 2078: [0, 1544, 2078],
 2110: [0, 1544, 2110],
 2344: [0, 6092, 2344],
 3429: [0, 7636, 3429],
 3541: [0, 1378, 3541],
 4048: [0, 1378, 4048],
 4203: [0, 1378, 4203],
 4689: [0, 7636, 4689],
 4976: [0, 1544, 4976],
 5467: [0, 7636, 5467],
 6092: [0, 6092],
 6106: [0, 1544, 6106],
 6145: [0, 1378, 6145],
 6733: [0, 1378, 6733],
 6769: [0, 1544, 6769],
 7012: [0, 14442, 7012],
 7191: [0, 6092, 7191],
 7297: [0, 6092, 7297],
 7379: [0, 1544, 7379],
 7443: [0, 1378, 7443],
 7465: [0, 7636, 7465],
 7636: [0, 7636],
 7798: [0, 14442, 7798],
 7851: [0, 14442, 7851],
 8117: [0, 14442, 8117],
 8140: [0, 7636, 8140],
 8302: [0, 1378

In [90]:
type(two_path)

dict

In [ ]:
features = preprocess_features(features)

In [ ]:
#print(features.shape)
features[1].shape
#2708*1433

In [ ]:
print(adj.toarray())

In [ ]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow as tf

from utils import *
from models import GCN, MLP

# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('dataset', 'pubmed', 'Dataset string.')  # 'cora', 'citeseer', 'pubmed'
flags.DEFINE_string('model', 'gcn', 'Model string.')  # 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 16, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 10, 'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

# Load data
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(FLAGS.dataset)

# Some preprocessing
features = preprocess_features(features)
if FLAGS.model == 'gcn':
    support = [preprocess_adj(adj)]
    num_supports = 1
    model_func = GCN
elif FLAGS.model == 'gcn_cheby':
    support = chebyshev_polynomials(adj, FLAGS.max_degree)
    num_supports = 1 + FLAGS.max_degree
    model_func = GCN
elif FLAGS.model == 'dense':
    support = [preprocess_adj(adj)]  # Not used
    num_supports = 1
    model_func = MLP
else:
    raise ValueError('Invalid argument for model: ' + str(FLAGS.model))

# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

# Create model
model = model_func(placeholders, input_dim=features[2][1], logging=True)

# Initialize session
sess = tf.Session()


# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], (time.time() - t_test)


# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []

# Construct feed dictionary
feed_dict = construct_feed_dict(features, support, y_train, train_mask, placeholders)
feed_dict.update({placeholders['dropout']: FLAGS.dropout})


# Train model
for epoch in range(FLAGS.epochs):

    t = time.time()
  

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation
    cost, acc, duration = evaluate(features, support, y_val, val_mask, placeholders)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and cost_val[-1] > np.mean(cost_val[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

# Testing
test_cost, test_acc, test_duration = evaluate(features, support, y_test, test_mask, placeholders)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))


In [ ]:
y_val.shape

In [ ]:
features